In [1]:
import base64
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema.messages import SystemMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from operator import itemgetter

In [2]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [3]:
model = ChatOpenAI(model='gpt-4o-mini')
embeddings = OpenAIEmbeddings()
parser = StrOutputParser()

In [4]:
loader = PyMuPDFLoader('ref_data/TWINGO/MR413X4416A000.pdf', extract_images=True)
pages = loader.load()
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)
retriever = vectorstore.as_retriever()

/home/andrew/Projects/TechData-RAG/.venv/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [5]:
SYSTEM_PROMPT = '''
You are a witty assistant that will use the chat history and the image 
provided by the user to answer its questions.

Use few words on your answers. Go straight to the point. Do not use any
emoticons or emojis. Do not ask the user any questions.

Be friendly and helpful. Show some personality. Do not be too formal.
'''

prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=SYSTEM_PROMPT),
        MessagesPlaceholder(variable_name='chat_history'),
        (
            'human',
            [
                {'type': 'text', 'text': '{prompt}'},
                {
                    'type': 'image_url',
                    'image_url': 'data:image/jpeg;base64,{image_base64}',
                },
            ],
        ),
    ]
)

In [7]:
chain = prompt_template | model | parser
chain = RunnableWithMessageHistory(
    chain,
    lambda _: ChatMessageHistory(),
    input_messages_key="prompt",
    history_messages_key="chat_history",
)

In [8]:
question = """
что нарисовано на этой схеме, приведи список: БЛОК-СХЕМА СИСТЕМЫ ЗАПУСКА ДВИГАТЕЛЯ И ЗАРЯДКИ АККУМУЛЯТОРНОЙ БАТАРЕИ?
"""
image = encode_image('ref_data/Starter image.jpeg')

In [11]:
(itemgetter("question") | retriever)

RunnableLambda(itemgetter('question'))
| VectorStoreRetriever(tags=['DocArrayInMemorySearch'], vectorstore=<langchain_community.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x7e9f29c7dba0>)

In [10]:
for s in chain.stream({"prompt": question, "image_base64": image},
                      config={"configurable": {"session_id": "unused"}}):
    print(s, end="", flush=True)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


KeyError: "Input to ChatPromptTemplate is missing variables {'context'}.  Expected: ['chat_history', 'context', 'image_base64', 'prompt'] Received: ['prompt', 'image_base64', 'chat_history']"